In [ ]:
'''
Single hidden layer neural network

A[1] = relu(W[1].T dot X + B[1])
A[2] = sigmoid(W[2].T dot A[1] + B[2])

where X[1]: (n_x, n_examples), W[1]: (n_x, n_hidden), B[1]: (n_hidden, n_examples)
and A[1]: (n_hidden, n_examples), W[2]: (n_hidden, n_y), B[2]: (n_y, n_examples)

Considering the n_examples can vary from training to testing, the biases will be broadcasted

Further inquiries:
Understand relation between input topologies and network architecture
'''

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage

In [ ]:
n_x = 12288
n_hidden = 4
n_y = 1
layer_sizes = [n_x, n_hidden, n_y]
n_layers = len(layer_sizes) - 1

In [ ]:
def relu(z):
    return max(0, z)

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:

def init_params(layer_sizes):

    params = dict()
    for l in range(1, len(layer_sizes)):
        params['W{layer}'.format(layer=l)] = np.random.randn(layer_sizes[l - 1],layer_sizes[l]) * np.sqrt(2/layer_sizes[l-1]) # He et al. initialization
        params['B{layer}'.format(layer=l)] = np.zeros((layer_sizes[l]))
    
    return params

def forward_prop(params, X, n_layers):
    
    cache = dict()
    cache['A0'] = X
    for l in range(1, n_layers + 1):
        Z_l = np.dot(params['W{layer}'.format(layer=l)].T, cache['A{prev_l}'.format(prev_l=l-1)]) + params['B{layer}'.format(layer=l)]
        cache['A{layer}'.format(layer=l)] = relu(Z_l) if l != n_layers else sigmoid(Z_l)
    return params, cache

def backward_prop():
    pass

def optimize():
    pass

In [ ]:
def predict():
    pass

def model():
    pass